In [419]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import warnings # Uyarılar
warnings.filterwarnings("ignore")

df=pd.read_csv('data.csv',encoding='unicode_escape').copy()

#
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [420]:
def check_df(dataframe):
    print("################ Shape ####################")
    print(dataframe.shape)
    print("############### Columns ###################")
    print(dataframe.columns)
    print("############### Types #####################")
    print(dataframe.dtypes)
    print("############### Head ######################")
    print(dataframe.head())
    print("############### Tail ######################")
    print(dataframe.tail())
    print("############### Describe ###################")
    print(dataframe.describe().T)

check_df(df)

################ Shape ####################
(541909, 8)
############### Columns ###################
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')
############### Types #####################
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object
############### Head ######################
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  Customer

Checking for any null entries

In [421]:
df.isnull().sum() #checks for empty entries

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [422]:

df.dropna(inplace=True) #removes empty entries, maintains the dataframe in the same variable

Get number of unique items in the dataset

In [423]:
df["Description"].nunique()

3896

Product types with their quantity numbers

In [424]:
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ANTIQUE RASPBERRY FLOWER EARRINGS        1
WALL ART,ONLY ONE PERSON                 1
GOLD/AMBER DROP EARRINGS W LEAF          1
INCENSE BAZAAR PEACH                     1
PINK BAROQUE FLOCK CANDLE HOLDER         1
Name: Description, Length: 3896, dtype: int64

Rank the 5 most ordered products from most to least

In [425]:
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False).head(5)


,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


Remove cancelled entries

In [426]:
df = df[~df["InvoiceNo"].str.contains("C", na=False)]

The total amount of each invoice and we create that column (just multiply 'Price' and 'Quantity')

In [427]:
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]


RFM Metrics

In [428]:
#Determining the analysis date for the recency 
df["InvoiceDate"].max()
today_date = dt.datetime(2011, 12, 11)
print(today_date)
print(pd.to_datetime("12/1/2010  8:26:00PM"))
rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (today_date - pd.to_datetime(x.max())).days,
                                     'InvoiceNo': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.columns=["Recency","Frequency","Monetary"]
rfm = rfm[rfm["Monetary"] > 0]
rfm.describe().T

2011-12-11 00:00:00
2010-12-01 20:26:00


,count,mean,std,min,25%,50%,75%,max
Recency,4338.0,139.663209,92.782756,1.00,76.000,110.000,196.50,374.00
Frequency,4338.0,4.272706,7.706221,1.00,1.000,2.000,5.00,210.00
Monetary,4338.0,2054.266460,8989.230441,3.75,307.415,674.485,1661.74,280206.02


Converting RFM Scores to Single Variable
- The nearest date gets 5 and the furthest date gets 1.


In [430]:
#Date from customer's last purchase.The nearest date gets 5 and the furthest date gets 1.

rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
# Total number of purchases.The least frequency gets 1 and the maximum frequency gets 5.
rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
#Total spend by the customer.The least money gets 1, the most money gets 5.
rfm["monetary_score"]= pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
CustomerID,,,,,,
12346.0,326,1,77183.60,1,1,5
12347.0,130,7,4310.00,3,5,5
12348.0,76,4,1797.24,4,4,4
12349.0,19,1,1757.55,5,1,4
12350.0,311,1,334.40,1,1,2


In [431]:
#RFM - The value of 2 different variables that were formed was recorded as a RFM_SCORE (recency and frequency)
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

Interpretation of Statistics of segments

In [434]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm[["segment", "Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","count","max"]).round()

Recency            Frequency            Monetary        \
                       mean count  max      mean count  max     mean count   
segment                                                                      
about_to_sleep        116.0   155  137       1.0   155    2    529.0   155   
at_Risk               215.0   763  374       3.0   763    5   1342.0   763   
cant_loose            184.0   119  373       8.0   119   34   3262.0   119   
champions              45.0   148   72       8.0   148   97   3601.0   148   
hibernating           259.0   853  374       1.0   853    2    488.0   853   
loyal_customers        96.0  1137  137      10.0  1137  210   5069.0  1137   
need_attention        114.0   143  137       2.0   143    3    869.0   143   
new_customers          38.0   303   72       1.0   303    1    396.0   303   
potential_loyalists    50.0   649   93       2.0   649    3    654.0   649   
promising              82.0    68   93       1.0    68    1    524.0    68   

                               
                          max  
segment                        
about_to_sleep         4874.0  
at_Risk              168472.0  
cant_loose            28337.0  
champions             65040.0  
hibernating           77184.0  
loyal_customers      280206.0  
need_attention         4056.0  
new_customers          6208.0  
potential_loyalists   12602.0  
promising              4367.0